In [1]:
import sys
import pygame
import string
from random import sample

pygame 2.1.2 (SDL 2.0.18, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
color_gray = (90, 90, 90)
color_lightgray = (240, 240, 240)
color_white = (255, 255, 255)
color_black = (0, 0, 0)
color_blue = (0, 0, 255)
color_red = (255, 0, 0)
color_green = (0, 255, 0)

button_r = 50
font_size = 30
borders_widht = 3
source = list(map(str, range(100)))
dimentions = 7
sample_range = (1,20)
n_ans_buttons = 7

In [3]:
print([(i,j) for i in range(3) for j in range(5)])

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4)]


In [4]:
class Game(object):
    
    def __init__(self, name, dimentions=5, ans_buttons=7, sample_range=(1,20)):
        self.name = name
        self.dimentions = dimentions
        self.ans_buttons = ans_buttons
        self.sample_range = sample_range
        self.score = 0
        self.total_score = 0
        
    def new_game(self):
        self.ingame = True
        self.score = 0
        self.time = 0
        self.quiz_line = 1
        self.quiz_hex = 0
        seed_line = sample(range(self.sample_range[0], self.sample_range[1]), self.dimentions)
        self.puzzle_table = [seed_line]
        while len(seed_line) > 1:
            seed_line = [i + j for i, j in zip(seed_line, seed_line[1:])]
            self.puzzle_table.append(seed_line)
        
 #       lines_to_draw[self.quiz_line].set_color(self.quiz_hex, color_red)
        ans_buttons.permutate(self.puzzle_table[self.quiz_line][self.quiz_hex])


    def update(self, score):
        self.score += score
        lines_to_draw[self.quiz_line].set_color(self.quiz_hex, color_lightgray)
        lines_to_draw[self.quiz_line].show_button(self.quiz_hex)
        if self.quiz_hex >= (len(self.puzzle_table[self.quiz_line]) - 1):
            self.quiz_hex = 0
            self.quiz_line +=1
        else:
            self.quiz_hex += 1
        if self.quiz_line > (len(self.puzzle_table) - 1):
            self.ingame = False
        else:
            lines_to_draw[self.quiz_line].set_color(self.quiz_hex, color_red)
            ans_buttons.permutate(self.puzzle_table[self.quiz_line][self.quiz_hex])

In [5]:
class Hextangle(object):
    
    def __init__(self, x, y, r, color_fill, color_border, color_text, text, shown=False):
        self.x = x
        self.y = y
        self.r = r
        self.color_fill = color_fill
        self.color_border = color_border
        self.text = smallfont.render(text , True , color_text)
        self.shown = shown
        
        self.points = [(x, y+r),
                       (x+0.87*r, y+0.5*r),
                       (x+0.87*r, y-0.5*r),
                       (x, y-r),
                       (x-0.87*r, y-0.5*r),
                       (x-0.87*r, y+0.5*r)]
    
    def draw(self):
        pygame.draw.polygon(scr, self.color_fill, self.points)
        pygame.draw.polygon(scr, self.color_border, self.points, width=borders_widht)
        if self.shown:
            scr.blit(self.text , (self.x - 15, self.y - font_size/2))
    
    def show(self):
        self.shown = True
    
    def set_color(self, color):
        self.color_fill = color

In [6]:
class Hextangle_line(object):
    
    def __init__(self, screen, n_line, y_line, button_r):
        self.n_line = n_line
        self.y_line = y_line
        self.button_r = button_r
        self.screen_width = screen.get_width()
        
    def create_buttons(self, buttons_text):
        self.leftmost_x = (self.screen_width - 1.73*self.button_r*(len(buttons_text)))/2 + 0.87*self.button_r
        self.buttons = [Hextangle(self.leftmost_x + i*self.button_r*1.73,
                               self.y_line,
                               self.button_r,
                               color_lightgray,
                               color_black,
                               color_black,
                               t,
                               (self.n_line == 0)) \
                        for i, t in enumerate(buttons_text)]
    
    def draw(self):
        for button in self.buttons:
            button.draw()
    
    def show_button(self, n):
        self.buttons[n].show()
    
    def set_color(self, n, color):
        self.buttons[n].set_color(color)

In [7]:
class Button(object):
    
    def __init__(self, x, y, height, widht, color_fill, color_border, color_text, text):
        self.x = x
        self.y = y
        self.height = height
        self.widht = widht
        self.color_fill = color_fill
        self.color_border = color_border
        self.text = avgfont.render(text , True , color_text)
    
    def draw(self):
        pygame.draw.rect(scr, self.color_fill, (self.x, self.y, self.widht, self.height))
        pygame.draw.rect(scr, self.color_border, (self.x, self.y, self.widht, self.height), width=borders_widht)
        scr.blit(self.text , (self.x + self.widht/4, self.y + self.height/5))
        
    def check_pressed(self, x, y):
        return (self.x <= x <= (self.x+self.widht)) and (self.y <= y <= (self.y+self.height))

In [8]:
class Ans_button_line(object):
    
    def __init__(self, screen, y, height, widht, n_buttons):
        self.n_buttons = n_buttons
        self.y = y
        self.height = height
        self.widht = widht
        self.n_buttons = n_buttons       
        self.distance = (screen.get_width() - n_buttons*widht)/(n_buttons+1)
        
    def permutate(self, answer, ans_range=(1,10)):
        self.answers = [answer + i for i in sample(list(range(ans_range[0], ans_range[1])) +\
                                                   list(range(-ans_range[1], -ans_range[0])), self.n_buttons-1)]
        self.correct_position = sample(range(self.n_buttons), 1)[0]
        self.answers.insert(self.correct_position, answer)
        self.buttons = [Button(self.distance + i*(self.distance + self.widht),
                       self.y,
                       self.height,
                       self.widht,        
                       color_lightgray,
                       color_black,
                       color_black,
                       str(t)) \
                for i, t in enumerate(self.answers)]
    
    def draw(self):
        for button in self.buttons:
            button.draw()
    
    def check_click(self, x, y):
        try:
            answer = [button.check_pressed(x, y) for button in self.buttons].index(True)
            return int(answer == self.correct_position)*2 - 1
        except ValueError:
            return 0

In [9]:
def handle_keyboard_events():
    if event.key == pygame.K_ESCAPE:
        print(game.score)

In [10]:
def handle_mouse_events():
    mouse_coord = pygame.mouse.get_pos()
    answer = ans_buttons.check_click(mouse_coord[0], mouse_coord[1])
    if answer != 0 and game.ingame:
        game.update(answer)

In [16]:
def drawing():
    scr.fill(color_gray)
    
    scr.blit(avgfont.render(f'Score: {game.score}', True , color_red) , (50, 50))
    
    for line in lines_to_draw:
        line.draw()
    
    try:
        ans_buttons.draw()
    except NameError:
        pass
    
    pygame.display.update()
    FPS_CLOCK.tick(30)

In [17]:
pygame.init()
smallfont = pygame.font.SysFont('Arial', 30)
avgfont = pygame.font.SysFont('Arial', 40)

scr = pygame.display.set_mode((1200, 1000))
FPS_CLOCK = pygame.time.Clock()

game = Game(name = 'test', dimentions=7, ans_buttons=7, sample_range=(1,20))
ans_buttons = Ans_button_line(scr, 800, 80, 150, game.ans_buttons)    
lines_to_draw = [Hextangle_line(scr, line, 700-1.5*button_r*line, button_r) \
                 for line in range(game.dimentions)]

game.new_game()

for dim, line in zip(game.puzzle_table, lines_to_draw):
    line.create_buttons(list(map(str, dim)))


    
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if event.type == pygame.KEYDOWN:
            handle_keyboard_events()
        if event.type == pygame.MOUSEBUTTONDOWN:
            handle_mouse_events()
    drawing()


SystemExit: 